<a href="https://colab.research.google.com/github/VallabhK/SentimentExtraction/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sentiment Extraction**
This is the program which parses the tweets from Twitter using python and other open source libraries

In [1]:
#Importing all the required libraries
import tweepy #Access twitter API 
from textblob import TextBlob #Process text data
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
url = 'https://raw.githubusercontent.com/VallabhK/SentimentExtraction/master/config.csv?token=AIB6MQJOQ7PJ52BYLLSIIK3A7BMIC'
config = pd.read_csv(url)

In [4]:
twitterApiKey = config['twitterApiKey'][0]
twitterApiKeySecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [5]:
#Create authentication object
authenticate = tweepy.OAuthHandler(twitterApiKey, twitterApiKeySecret)

#Set access token and secret
authenticate.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)

#Create the API object while passing in auth information
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [6]:
#Extract tweets
posts= api.user_timeline(screen_name ='BillGates', count =100, lang ='en', tweet_mode ='extended')

print("Show tweets \n")
for tweet in posts[0:5]:
  print(tweet.full_text + '\n')

Show tweets 

Our foundation joined the polio fight almost 15 years ago. And in all that time, I cannot think of a more important moment than right now. https://t.co/ZUyvSZyzau

I look forward to working with you and @EU_Commission on all of these important goals. Europe is a critical partner in creating a net-zero future and ensuring a healthy and equitable world for all. https://t.co/O16waKVVVo

RT @EIB: How can we:
💨 Boost #greentech growth?
🔽 Lower costs?
🪙 Help commercialise #sustainable projects faster?

President Hoyer met with…

Reaching net-zero emissions by 2050 means transforming our entire physical economy and accelerating innovation at an unprecedented pace. @Breakthrough Energy Catalyst will jumpstart this process. https://t.co/MAKM2lAyy0

RT @gatesfoundation: Support for polio eradication will not only help wipe out a disease, it will prepare countries against future health t…



In [7]:
#Create a dataframe with all the tweets
df =  pd.DataFrame([tweet.full_text for tweet in posts], columns =['Tweets'])

In [8]:
#Clean the tweets
def cleanText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # r - expression is a raw string
  text = re.sub(r'#','',text) #Remove # symbol
  text = re.sub(r'RT[\s]+','' ,text)
  text = re.sub(r'https?:\/\/\S+','', text)
  return text

In [9]:
#Cleaning the data frame
df['Tweets'] =df['Tweets'].apply(cleanText)
df

,Tweets
0,Our foundation joined the polio fight almost 1...
1,I look forward to working with you and _Commis...
2,: How can we:\n💨 Boost greentech growth?\n🔽 Lo...
3,Reaching net-zero emissions by 2050 means tran...
4,: Support for polio eradication will not only ...
...,...
95,"Thanks for inviting me on the podcast, ."
96,": The Weekly Planet: Lately, Bill Gates has be..."
97,"Thank you, _RDG. We have a lot of work ahead o..."
98,": At Breakthrough Energy, we don’t just think ..."
